#### *IBM Data Science Certification Capstone Project - authored by Julien Girault, data scientist scholar*
# Find your Favourite Tube Station before Moving to London!

## Project introduction

Many French people move from Paris to London to improve their English and live a new experience there.

But what is the best place they should choose for settling down?

When moving to a new city, one will likely choose their new address based on public transports: they will search the lines that can bring them quickly and directly from their new home to their office.

__"London tube" and " Paris métro" public transports are comparably big with approximately 400 stations__... So here is the question that comes next: how can one know the kind of surroundings they are likely to find around each station? How can they decide where to settle down? Which station should they prefer with so many possibilities?

This project will give French newcomers the opportunity to find the list of tube stations in London that best fit their taste, based on a comparison with something they already know: Paris' stations!

For each London tube station, we will expose a list of Paris stations that are similar.

To reach that goal, we will train an AI clustering model to find which tube stations are similar to each French station.


## Data Description

For this project, I downloaded the transports stations coordinates released by the public services for both London [1] and Paris [2].

In the main section of the project, I will use the Foursquare API with each station coordinates to explore the list of venues reachable within a five-minute walk.

It might be necessary to help manually with the matching of venues categories, for instance, to transform "pubs" into "bars", or change "bed & breakfast" to "hotel".

## References

[1] [London Tube Stations List (CSV)](https://www.whatdotheyknow.com/request/512947/response/1238210/attach/3/Stations%2020180921.csv.txt?cookie_passthrough=1),

[2] [Paris Transports Stations List website](https://data.iledefrance-mobilites.fr/explore/dataset/emplacement-des-gares-idf/table/?refine.mode=Metro)

